<a href="https://colab.research.google.com/github/NULabTMN/homework-2-gmac98/blob/main/PageRank.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PageRank

In this assignment, you will compute PageRank on a collection of 469,235 web sites using the iterative version of the PageRank algorithm described in class for sparse graphs (NOT the power method with explicit matrix multiplication).

Consider the following directed graph:

![A directed link graph](https://ccs.neu.edu/home/dasmith/courses/cs6200/pagerank.jpg)

We can represent this graph as a collection of nodes, here, ordered pairs of node index and node name:

In [1]:
small_nodes = [(0, 'A'),
              (1, 'B'),
              (2, 'C'),
              (3, 'D'),
              (4, 'E'),
              (5, 'F')]

and a collection of directed links, i.e., ordered pairs from source to target:

In [2]:
small_edges = [
  (0, 1),
  (0, 2),
  (0, 5),
  (1, 2),
  (1, 3),
  (1, 4),
  (1, 5),
  (2, 3),
  (2, 4),
  (3, 0),
  (3, 2),
  (3, 4),
  (3, 5),
  (4, 0),
  (5, 0),
  (5, 1),
  (5, 4)
]

We use integer identifiers for the nodes for efficiency. Note that, unlike this example, in a real web graph, not every page will have in-links, nor will every page have out-links.

## First Implementation and Test

\[10 points\] Implement the iterative PageRank algorithm. Test your code on the six-node example using the input representation given above.  Be sure that your code handles pages that have no in-links or out-links properly.  (You may wish to test on a few such examples.) In later parts of this assignment, depending on how you store the data, it may be convenient to use iterators rather than storing the data in memory.

In [3]:
# TODO: Implement PageRank, given nodes and edges, to start with a uniform
# distribution over nodes, run a fixed number of iterations, and
# return a distribution over nodes.
import math
import numpy as np
from collections import defaultdict

def page_rank_fixed_iter(nodes, edges, max_iterations=10):
    
    damping_factor = 0.85
    node_dict = dict(nodes)
    
    page_set = []
    dict1 = defaultdict(list)
    for i, j in edges:
        page_set.append(i)
        page_set.append(j)
        dict1[j].append(i)
    page_set = list(dict.fromkeys(page_set))
    inlink_dict = dict(dict1)
        
    for page in page_set:
        if page not in inlink_dict.keys():
            inlink_dict[page] = []

    outlink_count_dict = {}
    for page in page_set:
        outlink_count_dict[page] = 0

    for values in inlink_dict.values():
        for v in values:
            outlink_count_dict[v] += 1

    sink_set = []
    for k in outlink_count_dict.keys():
        if outlink_count_dict.get(k) == 0:
            sink_set.append(k)
    
    # page rank calculation
    N = len(page_set)
    final_rank_dict = {}
    pagerank_new = {}

    for page in page_set:
        final_rank_dict[page] = 1 / N

    iter = 0
    while iter < max_iterations:
        sink_rank = 0
        for page in sink_set:
            sink_rank += final_rank_dict[page]
        for page in page_set:
            pagerank_new[page] = (1 - damping_factor) / N
            pagerank_new[page] += damping_factor * sink_rank / N
            for i in inlink_dict[page]:
                pagerank_new[page] += damping_factor * final_rank_dict[i] / outlink_count_dict[i]
        final_rank_dict = pagerank_new
        iter += 1
      
    pagerank_sorted = sorted(final_rank_dict.items())
    return list((page[0],node_dict[page[0]],page[1]) for page in pagerank_sorted)

# Output PageRank on the toy graph at various points.
# Make sure your output has node number, name, and PageRank value.
print("Page Rank on toy graph at 1 iteration: \n", page_rank_fixed_iter(small_nodes, small_edges, 1))
print("Page Rank on toy graph at 10 iterations: \n",page_rank_fixed_iter(small_nodes, small_edges, 10))
print("Page Rank on toy graph at 100 iterations: \n",page_rank_fixed_iter(small_nodes, small_edges, 100))

Page Rank on toy graph at 1 iteration: 
 [(0, 'A', 0.24930555555555556), (1, 'B', 0.11944444444444445), (2, 'C', 0.14305555555555555), (3, 'D', 0.13125), (4, 'E', 0.2138888888888889), (5, 'F', 0.14305555555555555)]
Page Rank on toy graph at 10 iterations: 
 [(0, 'A', 0.25270193989927037), (1, 'B', 0.1395924623608689), (2, 'C', 0.15158942960414695), (3, 'D', 0.1190889058334471), (4, 'E', 0.18734563671089624), (5, 'F', 0.15158942960414695)]
Page Rank on toy graph at 100 iterations: 
 [(0, 'A', 0.2521271053751956), (1, 'B', 0.1393061853185386), (2, 'C', 0.1513064898667053), (3, 'D', 0.11890782257353921), (4, 'E', 0.18704590699931614), (5, 'F', 0.1513064898667053)]


## PageRank on Web Crawl Data

\[20 points\] Download and unpack a list of `.edu` websites and the links among them from the [Common Crawl](https://commoncrawl.org/2017/05/hostgraph-2017-feb-mar-apr-crawls/) open-source web crawl. For the sake of brevity, the data record links among websites, not web pages. The information for nodes and links is the same as the toy example above.

In [4]:
# If you're running on a machine (e.g., Windows) that doesn't have wget or gzip,
# feel free to comment this out and use a different set of commands to load
# the data.
!wget https://ccs.neu.edu/home/dasmith/courses/cs6200/vertices-edu.txt.gz
!gzip -df vertices-edu.txt.gz
!wget https://ccs.neu.edu/home/dasmith/courses/cs6200/edges-edu.txt.gz
!gzip -df edges-edu.txt.gz

--2022-02-28 03:41:52--  https://ccs.neu.edu/home/dasmith/courses/cs6200/vertices-edu.txt.gz
Resolving ccs.neu.edu (ccs.neu.edu)... 52.70.229.197
Connecting to ccs.neu.edu (ccs.neu.edu)|52.70.229.197|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3703486 (3.5M) [application/x-gzip]
Saving to: ‘vertices-edu.txt.gz’

vertices-edu.txt.gz 100%[===================>]   3.53M  --.-KB/s    in 0.09s   

2022-02-28 03:42:07 (38.9 MB/s) - ‘vertices-edu.txt.gz’ saved [3703486/3703486]

--2022-02-28 03:42:07--  https://ccs.neu.edu/home/dasmith/courses/cs6200/edges-edu.txt.gz
Resolving ccs.neu.edu (ccs.neu.edu)... 52.70.229.197
Connecting to ccs.neu.edu (ccs.neu.edu)|52.70.229.197|:443... connected.
HTTP request sent, awaiting response... 503 Service Temporarily Unavailable
2022-02-28 03:42:28 ERROR 503: Service Temporarily Unavailable.

gzip: edges-edu.txt.gz: No such file or directory


There should now be files `vertices-edu.txt` and `edges-edu.txt`.

In [5]:
# TODO: Process the raw data into the same format as the simple graph.
# You may create lists or iterators.
edges = []
with open('edges-edu.txt', 'r') as f:
    edges = [l.strip() for l in f.readlines()]
    
nodes = []
with open('vertices-edu.txt', 'r') as f:
    nodes = [l.strip() for l in f.readlines()]

Refine your implementation of PageRank to test for numerical convergence. Specificially, at each iteration, calculate the [perplexity](https://en.wikipedia.org/wiki/Perplexity) of the PageRank distribution, where perplexity is defined as 2 raised to the [Shannon entropy](https://en.wikipedia.org/wiki/Entropy_(information_theory)) of the PageRank distribution, i.e., $2^{H(PR)}$. (Recall that we defined entropy when talking about data compression.) The maximum perplexity of a PageRank distribution will therefore be the number of nodes in the graph.

At each iteration, check the _change_ in perplexity. If the change is less than some threshold, you can stop.


In [6]:
from math import log

def page_rank(nodes, edges, threshold=1):
    
    damping_factor = 0.85 
    final_rank_dict = {}
    pagerank_new = {}   
    perplexity = []

    page_set = []
    dict1 = defaultdict(list)
    for edge in edges:
        nodes = edge.split()
        from_, to_ = nodes[0],nodes[1]
        page_set.append(from_)
        page_set.append(to_)
        dict1[to_].append(from_)

    page_set = list(dict.fromkeys(page_set))
    inlink_dict = dict(dict1)
    for page in page_set:
        if page not in inlink_dict.keys():
            inlink_dict[page] = []

    outlink_count_dict = {}
    inlink_count_dict = {}
    for page in page_set:
        outlink_count_dict[page] = 0
        inlink_count_dict[page] = 0

    for values in inlink_dict.values():
        for v in values:
            outlink_count_dict[v] += 1

    sink_set = []
    for k in outlink_count_dict.keys():
        if outlink_count_dict.get(k) == 0:
            sink_set.append(k)

    for k in inlink_dict.keys():
        inlink_count_dict[k] = len(inlink_dict.get(k))

    source_set = []
    for i in inlink_count_dict:
        if inlink_count_dict.get(i) == 0 :
            source_set.append(i)
            
    # numerical convergence checking helper function
    def check_convergence(iter):
        perplexity.append(compute_perplexity(iter))
        if iter > 0:
            diff = abs(perplexity[iter] - perplexity[iter - 1])
            if diff < threshold:
                return True
            else:
                return False
        else:
            return False

    # perplexity calculation helper function
    def compute_perplexity(iter):
        H = 0
        for page in final_rank_dict.keys():
            H += final_rank_dict[page] * log( 1 / final_rank_dict[page], 2)
        perplexity_val = 2**H
        return perplexity_val
    
    
    # page rank calculation
    N = len(page_set)
    for page in page_set:
        final_rank_dict[page] = 1 / N

    iter = 0
    while not check_convergence(iter):
        print("Iteration : ",iter+1,", Perplexity :",compute_perplexity(iter))
        sink_rank = 0
        for page in sink_set:
            sink_rank += final_rank_dict[page]
        for page in page_set:
            pagerank_new[page] = (1 - damping_factor) / N
            pagerank_new[page] += damping_factor * sink_rank / N
            for i in inlink_dict[page]:
                pagerank_new[page] += damping_factor * final_rank_dict[i] / outlink_count_dict[i]
        # final_rank_dict = pagerank_new
        for page in page_set:
          final_rank_dict[page] = pagerank_new[page]
        iter += 1

    return final_rank_dict, inlink_count_dict, page_set, sink_set, source_set

# Run until perplexity changes by less than 1
print('Computing PageRank and change in perplexity')
final_rank_dict, inlink_count_dict, page_set, sink_set, source_set = page_rank(nodes, edges, 1)

Computing PageRank and change in perplexity
Iteration :  1 , Perplexity : 365475.00000684883
Iteration :  2 , Perplexity : 203521.1119383656
Iteration :  3 , Perplexity : 182231.54293486135
Iteration :  4 , Perplexity : 171348.75333868322
Iteration :  5 , Perplexity : 167989.13141405568
Iteration :  6 , Perplexity : 165876.4885006344
Iteration :  7 , Perplexity : 165026.42508863745
Iteration :  8 , Perplexity : 164384.73241848018
Iteration :  9 , Perplexity : 164081.16846741337
Iteration :  10 , Perplexity : 163831.13997000034
Iteration :  11 , Perplexity : 163704.27042062077
Iteration :  12 , Perplexity : 163588.75888346083
Iteration :  13 , Perplexity : 163527.57152784104
Iteration :  14 , Perplexity : 163468.05352149493
Iteration :  15 , Perplexity : 163434.78700437472
Iteration :  16 , Perplexity : 163401.30566470226
Iteration :  17 , Perplexity : 163381.54199576954
Iteration :  18 , Perplexity : 163361.3469764021
Iteration :  19 , Perplexity : 163348.81422893237
Iteration :  20 , 

## Link Analysis

\[20 points\] In this final section, you will compute some properties of the web-site graph you downloaded.

First, consider the _in-link count_ of a website, simply the number of web-sites pointing to it (including self-links). 

In [7]:
# TODO: List the document ID, domain name, and in-link count of the 60 websites with the highest in-link count

sorted_inlink_count_dict = sorted(inlink_count_dict.items(), key=lambda x: x[1], reverse = True)

websitelist = []
for n in nodes:
  val = n.split()
  websitelist.append(val)
website_dict = dict(websitelist)

total = 0			
for website in sorted_inlink_count_dict:
    if(total < 60):
      print('Document ID: ', website[0],', domain name: ', website_dict[website[0]],', in-link count: ',website[1])
      total += 1

Document ID:  185524 , domain name:  edu.mit.web , in-link count:  4388
Document ID:  278032 , domain name:  edu.stanford , in-link count:  4021
Document ID:  244433 , domain name:  edu.purdue.english.owl , in-link count:  3531
Document ID:  140443 , domain name:  edu.indiana , in-link count:  3339
Document ID:  237176 , domain name:  edu.princeton , in-link count:  3251
Document ID:  64587 , domain name:  edu.columbia , in-link count:  3123
Document ID:  465503 , domain name:  edu.yale , in-link count:  2804
Document ID:  418623 , domain name:  edu.utexas , in-link count:  2622
Document ID:  383763 , domain name:  edu.unc , in-link count:  2592
Document ID:  197698 , domain name:  edu.nap , in-link count:  2494
Document ID:  439637 , domain name:  edu.washington , in-link count:  2291
Document ID:  373442 , domain name:  edu.umich , in-link count:  2281
Document ID:  440674 , domain name:  edu.washington.depts , in-link count:  2276
Document ID:  148945 , domain name:  edu.jhu.muse , 

Then, use the PageRank values compute by your second implementation. Note that some websites will have both a high in-link count and PageRank.

In [8]:
# TODO: List the document ID, domain name, and PageRank of the 60 websites with the highest PageRank.
pagerank_sorted = sorted(final_rank_dict.items(), key=lambda x: x[1], reverse = True)

total = 0
for website in pagerank_sorted:
    if(total < 60):
      print('Document ID: ', website[0], ', domain name: ', website_dict[website[0]], ', PageRank: ',website[1])
      total += 1

Document ID:  185524 , domain name:  edu.mit.web , PageRank:  0.0010322121730479438
Document ID:  465503 , domain name:  edu.yale , PageRank:  0.0007792600322022543
Document ID:  318278 , domain name:  edu.ucdavis.cas , PageRank:  0.0007138379243956912
Document ID:  278032 , domain name:  edu.stanford , PageRank:  0.0006612208801239564
Document ID:  136464 , domain name:  edu.illinois , PageRank:  0.0006447950534319456
Document ID:  319795 , domain name:  edu.ucdavis.ice.code , PageRank:  0.0006070516209454314
Document ID:  237176 , domain name:  edu.princeton , PageRank:  0.0005583215710547086
Document ID:  284517 , domain name:  edu.stanford.web , PageRank:  0.0005280978774652548
Document ID:  383763 , domain name:  edu.unc , PageRank:  0.0005246428403445298
Document ID:  64587 , domain name:  edu.columbia , PageRank:  0.000497604257700617
Document ID:  449738 , domain name:  edu.wisc , PageRank:  0.0004851902653943506
Document ID:  60975 , domain name:  edu.colorado , PageRank:  0.0

Finally, compute some summary statistics on this dataset.

In [9]:
# TODO: Compute:
# - the proportion of websites with no in-links (i.e., source nodes);
print('Proportion of websites with no in-links (i.e., source nodes) : ',len(source_set)/len(page_set))

# - the proportion of websites with no out-links (i.e., sink nodes);
print('Proportion of websites with no out-links (i.e., sink nodes) : ',len(sink_set)/len(page_set))

# - the proportion of websites whose PageRank is higher than the initial uniform distribution.
N = len(page_set)
count = 0
for website in pagerank_sorted:
  if(website[1] < 1 / N):
    count += 1
print('Proportion of websites whose PageRank is higher than the initial uniform distribution : ', count / N)

Proportion of websites with no in-links (i.e., source nodes) :  0.05188316574321089
Proportion of websites with no out-links (i.e., sink nodes) :  0.5001299678500581
Proportion of websites whose PageRank is higher than the initial uniform distribution :  0.8356987482043915
